In [1]:
#from exploringShipLogbooks.classification import LogbookClassifier

import numpy as np
import pandas as pd
import exploringShipLogbooks.wordCount as wc
from sklearn import preprocessing
from sklearn.naive_bayes import MultinomialNB

#import rpy2
#from rpy2.robjects import pandas2ri
#pandas2ri.activate()

from exploringShipLogbooks.basic_utils import extract_logbook_data
from exploringShipLogbooks.basic_utils import isolate_columns
#from exploringShipLogbooks.basic_utils import MultiColumnLabelEncoder
from exploringShipLogbooks.basic_utils import encode_data_df
from exploringShipLogbooks.basic_utils import clean_data

from exploringShipLogbooks.config import *


# Load and clean data
### Load CLIWOC ship logs

In [18]:
# extract data from zip file
cliwoc_data = extract_logbook_data('CLIWOC15.csv')

# extract logs that mention slaves
mentions_slaves = wc.count_key_words(cliwoc_data, text_columns, slave_words)
slave_index = mentions_slaves[(mentions_slaves['ContainsKeyWord'] != 0)].index
del mentions_slaves

# remove undesired columns
cliwoc_data = isolate_columns(cliwoc_data, desired_columns)

# clean data (make all same case)
cliwoc_data = clean_data(cliwoc_data)

# split log data into data that mentions slaves and data that does not
cliwoc_slave_logs = cliwoc_data.loc[slave_index.values]

# ****arbitrarily using first 1000 logs as "no slave" training data.
# need to figure out what to put here.
no_slave_index = range(1000)
cliwoc_no_slave_logs = cliwoc_data.loc[no_slave_index]

# remaining data is unclassified 
cliwoc_unclassified_logs = cliwoc_data.drop(slave_index)
cliwoc_unclassified_logs = cliwoc_data.drop(no_slave_index)

C:\Users\Emma\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3006: DtypeWarning: Columns (5,6,7,8,11,13,18,19,23,24,25,26,28,29,30,34,35,38,43,44,46,73,77,81,82,84,85,87,88,94,96,97,98,99,111,114,116,119,120,122,124,125,127,129,131,133,135,137,140) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


### Load Slave Voyages data

In [ ]:
filename = './exploringShipLogbooks/data/tastdb-exp-2010.sav'
slave_voyage_logs = rpy2.robjects.r('foreign::read.spss("%s", to.data.frame=TRUE)' % filename)

slave_voyage_desired_cols = ['portdep', 'portret', 'shipname', 'rig', 'national', 'yeardep']
slave_voyage_logs = isolate_columns(slave_voyage_logs, slave_voyage_desired_cols)

slave_voyage_logs.columns = ['ShipName', 'Nationality', 'ShipType', 'VoyageFrom', 'VoyageTo', 'Year']

slave_voyage_logs = clean_data(slave_voyage_logs)

# Encode data for classification

In [19]:
cliwoc_no_slave_logs = encode_data_df(cliwoc_no_slave_logs)
cliwpc_slave_logs = encode_data_df(cliwoc_slave_logs)
cliwoc_unclassified_logs = encode_data_df(cliwoc_unclassified_logs)

MemoryError: 

# Format data for classifier

# Fit training data to classifier

In [23]:

classifier = MultinomialNB()
classifier.fit(original_data, original_classes)

MultinomialNB(alpha = 1.0, class_prior = None, fit_prior = True)
prediction = classifier.predict(original_data)

print('Predictions for original data',
      '(should be same as original classes): \n', prediction)

new_data = np.random.randint(5, size = (1,100))
prediction = classifier.predict(new_data)

print('\n Predictions for new data (random)'
      ', should change for different runs: \n', prediction)

Predictions for original data (should be same as original classes): 
 [1 2 3 4 5 6]

 Predictions for new data (random), should change for different runs: 
 [2]
